### Run in collab
<a href="https://colab.research.google.com/github/racousin/rl_introduction/blob/master/notebooks/3_Temporal_Difference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install swig==4.2.1
!apt-get install xvfb
!pip install box2d-py==2.3.8
!pip install gymnasium[box2d,atari,accept-rom-license]==0.29.1
!pip install pyvirtualdisplay==3.0
!pip install opencv-python-headless
!pip install imageio imageio-ffmpeg
!git clone https://github.com/racousin/rl_introduction.git > /dev/null 2>&1

In [4]:
# from rl_introduction.rl_introduction.tools import Agent, plot_values_lake, policy_evaluation, value_iteration, discount_cumsum, MyRandomAgent, run_experiment_episode_train
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

# 3 Q_learning

### Objective
Implement and train a Q-Learning agent to interact with and learn from the 'FrozenLake-v1' environment without a known model.

### Experiment Setup: Evaluate and Train Your Agent

`run_experiment_episode_train` is the core function you will use for agent-environment interaction and learning:

In [9]:
env = gym.make('FrozenLake-v1')

def run_experiment_episode_train(env, agent, nb_episode, is_train=True):
    rewards = np.zeros(nb_episode)
    for i in range(nb_episode):
        state = env.reset()[0]
        done = False
        rews = []
        while not done:
            action = agent.act(state)
            current_state = state
            state, reward, done, info, _ = env.step(action)
            if is_train:
                agent.train(current_state, action, reward, state, done)
            rews.append(reward)
        rewards[i] = sum(rews)
        print(f'Episode: {i} - Cumulative Reward: {rewards[i]}')
    return rewards


In [10]:
class Agent:
    def __init__(self, env):
        self.env = env
    def act(self, state):
        action = env.action_space.sample()
        return action
    def train(self, current_state, action, reward, state, done):
        pass

In [11]:
demo_agent = Agent(env)
run_experiment_episode_train(env, demo_agent, nb_episode=10, is_train=True)

AttributeError: 'FrozenLakeEnv' object has no attribute 'observation_state'

**Exercise 1:** Initialize Q-Learning Agent

**Task 1a:** Initialize the `Agent` class with a Q-table filled with random values. The Q-table should have dimensions corresponding to the environment's state and action space sizes.

**Task 1b:** Create a function, `get_epsilon_greedy_action_from_Q_s`, that chooses an action based on an epsilon-greedy strategy or the argmax of Q for the current state.

**Task 1c:** Update the Agent class's act function to utilize `get_epsilon_greedy_action_from_Q_s` for action selection.

**Task 1d:** Implement the Q-learning update formula in the Agent class's train method.

$Q(S_t,A_t) \leftarrow Q(S_t,A_t)+ \alpha(R_{t+1}+\gamma \max_a Q(S_{t+1},a)−Q(S_t,A_t))$

**Exercise 2:** Train and Evaluate the Agent

**Task 2a:** Run 100 training episodes with the Q-learning agent and collect the rewards.

**Task 2b:** Plot the cumulative reward for each training episode.

**Question 1:**

How can we improve the learning convergence of our Q-learning agent?

In [8]:
#Done: get epsilon greedy policy
def get_epislon_greedy_action_from_q(Q_s,epsilon):
    if np.random.rand() > epsilon:
        return np.argmax(Q_s)
    else:
        return np.random.randint(len(Q_s))

In [ ]:
#Done: write Q learning update
class Agent():
    def __init__(self, env, gamma = .99, epsilon = .1, alpha = .01):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.alpha = alpha
        self.q = np.ones([self.env.observation_space.n, self.env.action_space.n]) / self.env.action_space.n
    def act(self, state):
        action = get_epislon_greedy_action_from_q(self.q[state], self.epsilon)
        return action
    def qsa_update(self, state, action, reward, next_state, done): 
        target = reward + (0 if done else self.gamma * np.max(self.q[next_state]))
        td_error = target - self.q[state, action]
        self.q[state, action]  += self.alpha * td_error
    def train(self, current_state, action, reward, next_state, done):
        self.qsa_update(current_state, action, reward, next_state, done)

In [ ]:
q_agent = Agent(env)
rewards = run_experiment_episode_train(env, q_agent, 5000)
fig,ax = plt.subplots(figsize=(10,10))
ax.plot(rewards,'+')
ax.set_title('cumulative reward per episode - q_agent')

In [ ]:

#Done: write Q learning update
class DecayAgent():
    def __init__(self, env, gamma = .99, epsilon = .1, alpha = .01, epsilon_decay_rate=.05):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.alpha = alpha
        self.initial_epsilon = epsilon
        self.episode = 0
        self.min_epsilon = 0
        self.epsilon_decay_rate = epsilon_decay_rate
        self.q = np.ones([self.env.observation_space.n, self.env.action_space.n]) / self.env.action_space.n
    def epsilon_decay_exponential(self):
        self.epsilon = self.initial_epsilon * (self.decay_rate ** self.episode)
        return max(self.min_epsilon, self.epsilon)
    def act(self, state):
        action = get_epislon_greedy_action_from_q(self.q[state], self.epsilon)
        self.episode += 1
        return action
    def qsa_update(self, state, action, reward, next_state, done): 
        target = reward + (0 if done else self.gamma * np.max(self.q[next_state]))
        td_error = target - self.q[state, action]
        self.q[state, action]  += self.alpha * td_error
    def train(self, current_state, action, reward, next_state, done):
        self.qsa_update(current_state, action, reward, next_state, done)

In [ ]:
# Assume 'env' is the environment, 'policy_random' is a random policy, and 'policy_optimal' is the optimal policy obtained from policy iteration

# Initialize agents with random and optimal policies
decay_agent = DecayAgent(env)
agent = Agent(env)

# Run experiments for each agent
rewards_decay_agent = run_experiment_episode_train(env, decay_agent, 5000)
rewards_agent = run_experiment_episode_train(env, agent, 5000)

# Visualization
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(rewards_agent, 'o')
plt.title('Cumulative Reward per Episode - decay_agent')
plt.xlabel('Episode')
plt.ylabel('Cumulative Reward')

plt.subplot(1, 2, 2)
plt.plot(rewards_agent, 'o')
plt.title('Cumulative Reward per Episode - agent')
plt.xlabel('Episode')

plt.tight_layout()
plt.show()


In [ ]:
exp_render({"name":'FrozenLake-v1', "fps":2, "nb_step":30, "agent": decay_agent})

# train/test your agent in other discrete action-space env

In [ ]:
env = gym.make('Blackjack-v1')
env = gym.make('Taxi-v3')

In [ ]:
your_agent = 
nb_episode = 10
run_experiment_episode_train(env, your_agent, nb_episode, is_train=False)